<a href="https://colab.research.google.com/github/mohamedGaber2004/langchainPractiseProjects/blob/main/langchainPractise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U langchain langchain-openai langchain-chroma langchain-core langchain-community langchain_huggingface pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.7 MB/s eta 0:

In [1]:
from google.colab import userdata

openaiKey = userdata.get('openai')
base_url = userdata.get('openRouterBaseUrl')

In [3]:
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage , AIMessage , SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.runnables import chain , RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [4]:
loader = PyPDFLoader("/content/drive/MyDrive/My CV/Mohamed-Gaber-Sayed-CVL4.pdf")
docs = loader.load()

In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300 ,
    chunk_overlap = 50 ,
)

In [7]:
splitted = splitter.split_documents(docs)

In [8]:
vector_store = Chroma.from_documents(splitted)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 36.8MiB/s]


In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
retriever = vector_store.as_retriever(
    embdddings = embeddings ,
    search_kwargs = {"k":3} ,
    search_type = "similarity"
)

In [11]:
prompt = """
You are a heplful Assistant ,
you are given this pdf inforamtion {pdfContext}
and help me to identify any thing the user will query it
by this query {query}
"""

In [12]:
prompt_template = ChatPromptTemplate.from_template(prompt)

In [13]:
llm = ChatOpenAI(
    model = "openai/gpt-oss-20b:free" ,
    base_url = base_url ,
    api_key= openaiKey
)

In [14]:
def getContext (docs) :
  return "\n\n".join([d.page_content for d in docs])

In [15]:
conversation = ({"pdfContext" : retriever | getContext  , "query" : RunnablePassthrough()}
| prompt_template
| llm
| StrOutputParser()
)

In [16]:
res = conversation.invoke("what is person's skills?")

In [17]:
print(res)

**Skills Profile (inferred from the provided PDF)**  

| Category | Skills & Tools | Evidence |
|----------|----------------|----------|
| **Programming & Libraries** | • Scikit‑learn <br>• TensorFlow <br>• PyTorch <br>• Pandas | Mentioned directly in the bio. |
| **Database** | • MySQL | Listed under “MySQL” heading. |
| **Machine Learning & Deep Learning** | • Supervised & unsupervised learning <br>• Neural network design <br>• Convolutional neural networks (ASL classification) | ASL Alphabet Classification project (99 % accuracy). |
| **Natural Language Processing (NLP)** | • NLP techniques & pipelines | Completed “NLP Specialization – Coursera”. |
| **Data‑Analysis & Feature Engineering** | • Exploratory data analysis <br>• Data cleaning & preprocessing | Stated “experience in data analysis using Pandas”. |
| **Cloud / Platform** | • NVIDIA Deep Learning platform (Deep Learning Specialization) | Listed as a course. |
| **Knowledge & Certification** | • AI Fundamentals (Instant) <br

**Skills Profile (inferred from the provided PDF)**  

| Category | Skills & Tools | Evidence |
|----------|----------------|----------|
| **Programming & Libraries** | • Scikit‑learn <br>• TensorFlow <br>• PyTorch <br>• Pandas | Mentioned directly in the bio. |
| **Database** | • MySQL | Listed under “MySQL” heading. |
| **Machine Learning & Deep Learning** | • Supervised & unsupervised learning <br>• Neural network design <br>• Convolutional neural networks (ASL classification) | ASL Alphabet Classification project (99 % accuracy). |
| **Natural Language Processing (NLP)** | • NLP techniques & pipelines | Completed “NLP Specialization – Coursera”. |
| **Data‑Analysis & Feature Engineering** | • Exploratory data analysis <br>• Data cleaning & preprocessing | Stated “experience in data analysis using Pandas”. |
| **Cloud / Platform** | • NVIDIA Deep Learning platform (Deep Learning Specialization) | Listed as a course. |
| **Knowledge & Certification** | • AI Fundamentals (Instant) <br>• ML Specialization (Coursera – Andrew Ng) <br>• Data Fundamentals (IBM – Skill‑Up) | All courses/credentials listed. |
| **Soft Skills** | • Communication <br>• Problem Solving <br>• Critical Thinking <br>• Collaboration <br>• Teamwork <br>• Time Management <br>• Continuous Learning | All explicit in the “Soft Skills” section. |
| **Books / Learning Resources** | • “Hands‑On Machine Learning with Scikit‑Learn, Keras, and TensorFlow” (Géron) | Indicates depth of self‑study and applied knowledge. |

### Quick Summary
- **Technical acumen:** MySQL, Python libraries (scikit‑learn, TensorFlow, PyTorch, Pandas), deep learning & NLP expertise, cloud‑based DL tools, robust data‑analysis skills.
- **Professional achievements:** A 99 % accurate ASL alphabet classifier & contributions to several high‑impact ML courses.
- **Soft‑skill repertoire:** Strong communicator, effective problem solver, critical thinker, collaborative team player, adept at time management, and committed to continuous learning.

This composite skill set positions the individual as a well-rounded data‑science / machine‑learning engineer ready for complex, team‑based projects.

In [18]:
for s in conversation.stream("what is person's skills?") :
  print(s , end="" , flush=True)

**Person’s skillset (as inferred from the PDF contents)**  

| Category | Skills / Experience |
|----------|---------------------|
| **Soft Skills** | • Communication<br>• Problem‑Solving<br>• Critical Thinking<br>• Collaboration<br>• Teamwork<br>• Time Management<br>• Continuous Learning |
| **Technical / Domain Knowledge** | • **Database**: MySQL (basic proficiency)<br>• **Machine Learning / Data Analysis**:<br> • Scikit‑learn<br> • Pandas<br> • TensorFlow<br> • PyTorch<br> • Deep‑Learning (e.g., convolutional networks for ASL alphabet classification – 99% accuracy)<br>• **Natural Language Processing** (NLP specializations)<br>• **Computer Vision / Deep Learning** (NVIDIA Deep Learning Specialization)<br>• **AI fundamentals** (e.g., “Artificial Intelligence – INSTANT” program) |
| **Projects** | • ASL Alphabet Classification – Deep Learning (Accuracy: 99%)<br>• (Any other highlighted project from the PDF) |
| **Education / Certifications** | • Coursera: Machine Learning (Andrew Ng)<b